In [6]:
import pandas as pd
import pyodbc as p

In [7]:
def limit10_fn(conn,db_name, table_name):
    query="""select * from {db_name}.{table_name} limit 10""".format(db_name=db_name,table_name=table_name)
    return pd.read_sql(query,conn)

This is the raw data script:

In [8]:
sql_query="""SELECT c.scheme_fk,scheme_name, ms_fk, c.dep_fk as dependent_no,upper(dep_type) as dep_type,gender,first_name,surname,birth_date, claim_fk, claim_date, pr_fk as practice_no,pr_descr as doctor_name, pr_type,pr_type_descr, pr_group_descr, tariff_fk, claim_type, c.claim_code,description as claim_code_description ,icd10_fk,icd10_descr, three_letter_level as icd10_group_description, auth_fk, units, pmb, (Case WHEN c.claim_code = 'K' THEN 'Y' WHEN c.claim_code IN ('C','P') AND c.claim_code = '80' THEN 'Y' WHEN pr_type = '62' AND c.claim_code = '31' THEN 'Y' WHEN pr_type IN ('87','88','90') THEN 'Y' ELSE 'N' END) as In_Hospital_Indicator,

claimed_amount, benefit_amount,

(case when (claimed_amount > 0 AND benefit_amount = 0) and paid_date is not null then 'Rejected' else case when ((claimed_amount > 0 AND benefit_amount > 0)) and paid_date is not null then 'Paid' else case when claimed_amount> 0 and benefit_amount > 0 and paid_date is null and assess_date is not null then 'Pending' else 'Unpaid' end end end ) as "ClaimsStatus",rcvd_date, assess_date, paid_date,tariff_amount, discount,owes, override, gen_no, suspend_until, suspended, assessed_datetime FROM mipbi_dbo.tf_claim_head_curr c left join mipst_dbo.tsd_ccdesc cc on c.claim_code=cc.claim_code and scheme_fk=scheme_code::int left join mipst_dbo.tsd_icd10 on icd10_fk=icd10_pk left join mipst_dbo.tsd_practice on pr_fk=pr_pk left join mipst_dbo.tsd_provider on pr_type_pk=pr_type left join mipbi_dbo.td_beneficiary b on ms_pk=ms_fk and b.scheme_fk=c.scheme_fk and b.dep_fk=c.dep_fk where claim_date>=20200101;"""

Here we connect to the __kx_bi__ postgres database and execute the above query to generate the EAP raw data set:

In [9]:
conn_str = (
    "DRIVER={PostgreSQL ODBC Driver(UNICODE)};"
    "DATABASE=kx_bi;"
    "UID=postgres;"
    "PWD=postgres;"
    "SERVER=172.31.244.44;"
    "PORT=5432;"
    )

conn = p.connect(conn_str)
# cursor = conn.cursor()

In [10]:
eapraw_df = pd.read_sql(sql_query,conn)

In [11]:
eapraw_df

,scheme_fk,scheme_name,ms_fk,dependent_no,dep_type,gender,first_name,surname,birth_date,claim_fk,...,assess_date,paid_date,tariff_amount,discount,owes,override,gen_no,suspend_until,suspended,assessed_datetime
0,1,ANGLO MEDICAL SCHEME - VALUE CARE,1684087,0,M,M,SIYANDA SIVELELE,MACFELLEN,19991028.0,008222,...,20200716,20200729.0,770.8,0.0,0.0,Aa,200SFL3M,None,N,2020-07-16 13:01:15.305
1,1,ANGLO MEDICAL SCHEME - VALUE CARE,1506745,0,M,F,NOSIPHO PROUD,HLANGEBI,19951125.0,046079,...,20200508,20200520.0,401.1,0.0,0.0,None,200S9YSS,None,N,2020-05-08 09:35:59.393
2,1,ANGLO MEDICAL SCHEME - VALUE CARE,1613989,1,C,F,MCKENZIE MADISON,ANDREWS,20140311.0,005763,...,20200124,20200205.0,401.1,0.0,0.0,None,200S38HM,None,N,2020-01-24 09:53:52.314
3,1,ANGLO MEDICAL SCHEME - VALUE CARE,1450727,2,C,M,KAMOGELO,SIBEKO,20070624.0,006013,...,20201002,20201013.0,378.5,0.0,0.0,None,200SKX2X,None,N,2020-10-02 19:22:48.744
4,1,ANGLO MEDICAL SCHEME - VALUE CARE,1684086,0,M,M,MJOZI JOHANNES,NGWENYA,19550303.0,005867,...,20200507,20200520.0,401.1,0.0,0.0,None,200S9WYY,None,N,2020-05-07 09:36:30.986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754597,140,FLEXICARE CORP,504960,0,M,F,BUSISIWE GCINILE,NTOMBELA,19800606.0,047533,...,20210204,20210216.0,163.2,0.0,0.0,None,210STTL5,None,N,2021-02-04 15:16:45.797
754598,140,FLEXICARE CORP,504960,0,M,F,BUSISIWE GCINILE,NTOMBELA,19800606.0,047533,...,20210204,20210216.0,163.2,0.0,0.0,None,210STTL6,None,N,2021-02-04 15:16:45.799
754599,140,FLEXICARE CORP,504960,0,M,F,BUSISIWE GCINILE,NTOMBELA,19800606.0,047533,...,20210204,20210216.0,163.2,0.0,0.0,None,210STTL2,None,N,2021-02-04 15:16:45.79
754600,140,FLEXICARE CORP,504960,0,M,F,BUSISIWE GCINILE,NTOMBELA,19800606.0,047533,...,20210204,20210216.0,163.2,0.0,0.0,None,210STTL4,None,N,2021-02-04 15:16:45.795
